# Plot the Evolution of Interconnection Temperature and Loads During Heat Waves


In [ ]:
# Start by importing the packages we need:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import timedelta
from sklearn import datasets, linear_model


## Set the Directory Structure

In [ ]:
# Identify the top-level directory and the subdirectory where the data will be stored:
load_data_input_dir =  '/Users/burl878/Documents/IMMM/Data/TELL/Analysis/Interconnection_Load_Time_Series/'
temp_data_input_dir =  '/Users/burl878/Documents/IMMM/Data/TELL/Analysis/Interconnection_Meteorology_Time_Series/'
image_output_dir =  '/Users/burl878/Documents/Travel/2023_AGU_Fall_Meeting/Poster/Figures/'


## Merge the Temperature and Load Datasets Together


In [ ]:
# Define a function to process the time series of hourly load and temperature for each interconnection:
def process_merged_time_series(load_data_input_dir: str, temp_data_input_dir: str, ics_to_plot: str, scenario: str, start_year: int, end_year: int):
    
    # Read in the interconnection load data:
    load_df = pd.read_csv(load_data_input_dir + scenario + '_Load_Time_Series.csv')
    
    # Set the start and end years to loop over:
    if scenario == 'historic':
       weather_scenario = scenario
    else:
       weather_scenario = scenario.split('_', 1)[0]
    
    # Loop over the years of weather data:
    for year in range(start_year, end_year, 1):
        # Read in the interconnection weather data:
        temp_df = pd.read_csv(temp_data_input_dir + weather_scenario + '/Interconnection_Meteorology_' + str(year) + '.csv')
        
        # Merge the dataframes together based on common times and interconnection labels:
        merged_df = pd.merge(temp_df, load_df, on=['Time_UTC', 'Interconnection'], how='left')
        
        # Store the output in a new dataframe:
        if year == start_year:
           output_df = merged_df
        else:
           output_df = pd.concat([output_df, merged_df])
    
    # Subset to just the interconnection you want to plot:
    ics_df = output_df.loc[(output_df['Interconnection'] == ics_to_plot)].reset_index()
    
    # Only keep the columns that are needed:
    ics_df = ics_df[['Time_UTC', 'Load_MWh', 'Percentile_Load', 'Normalized_Load', 'T2']].copy()
    
    # Convert the temperature from Kelvin to Fahrenheit:
    ics_df['T2'] = (1.8 * (ics_df['T2'] - 273)) + 32
    
    return ics_df


In [ ]:
# Test the function:
ics_df = process_merged_time_series(load_data_input_dir = load_data_input_dir, 
                                    temp_data_input_dir = temp_data_input_dir,
                                    ics_to_plot = 'WECC',
                                    scenario = 'rcp85hotter_ssp3',
                                    start_year = 2060,
                                    end_year = 2100)

ics_df


## Process and Plot the Historical Heat Wave Load Time Series


In [ ]:
# Define a function to process the load time series about the hottest hour of the year:
def process_heat_wave_load_time_series(load_data_input_dir: str, temp_data_input_dir: str, ics_to_plot: str, scenario: str, start_year: int, end_year: int):
    
    # Process the data using the function created above:
    ics_df = process_merged_time_series(load_data_input_dir = load_data_input_dir, 
                                        temp_data_input_dir = temp_data_input_dir,
                                        ics_to_plot = ics_to_plot,
                                        scenario = scenario,
                                        start_year = start_year,
                                        end_year = end_year)
    
    # Set the 'Time_UTC' value as a datetime variable:
    ics_df['Time_UTC'] = pd.to_datetime(ics_df['Time_UTC'])
    
    # Set the time variable as an index:
    ics_df.index = pd.to_datetime(ics_df['Time_UTC'])
    
    # Compute the daily minimum and maximum temperature using resampling:
    temp_df = ics_df.resample('D')['T2'].agg(['min', 'max']).reset_index()
        
    # Rename the variables for consistency:
    temp_df.rename(columns={'Time_UTC': 'Day', 'min': 'T2_Min', 'max': 'T2_Max'}, inplace=True)
        
    # Compute the daily minimum and maximum load using resampling:
    load_df = ics_df.resample('D')['Load_MWh'].agg(['min', 'max']).reset_index()    
        
    # Rename the variables for consistency:
    load_df.rename(columns={'Time_UTC': 'Day', 'min': 'Load_Min_MWh', 'max': 'Load_Max_MWh'}, inplace=True)
    
    # Merge the dataframes together based on common times and interconnection labels:
    merged_df = pd.merge(temp_df, load_df, on=['Day'], how='left')
    
    # Add a column with the year values to be used in grouping:
    merged_df['Year'] = merged_df['Day'].dt.year
    
    # Loop over the years from the 1980 to 2019:
    for year in range(start_year, end_year, 1):
        # Subset to just the year in question:
        met_df = merged_df.loc[(merged_df['Year'] == year)].copy()
    
        # Find hottest hour of the year:
        hottest_day = met_df['Day'].loc[met_df['T2_Max'].idxmax()]
        hottest_day_load = met_df['Load_Max_MWh'].loc[met_df['T2_Max'].idxmax()]
                
        # Normalize the loads based on the load at the hottest hour of the year:
        met_df['Load_Norm'] = met_df['Load_Max_MWh'].div(hottest_day_load)
        
        # Subset the full dataframe to a 28-day window around the hottest hour:
        hottest_load_subset_df = pd.DataFrame(met_df['Load_Norm'].loc[(met_df['Day'] >= (pd.to_datetime(hottest_day) - timedelta(days = 14))) 
                                                                    & (met_df['Day'] <= (pd.to_datetime(hottest_day) + timedelta(days = 14)))].reset_index(drop=True))
        hottest_temp_subset_df = pd.DataFrame(met_df['T2_Max'].loc[(met_df['Day'] >= (pd.to_datetime(hottest_day) - timedelta(days = 14))) 
                                                                 & (met_df['Day'] <= (pd.to_datetime(hottest_day) + timedelta(days = 14)))].reset_index(drop=True))
        
        # Rename the column to reflect the year:
        hottest_load_subset_df.rename(columns={'Load_Norm': str(year)}, inplace=True)
        hottest_temp_subset_df.rename(columns={'T2_Max': str(year)}, inplace=True)
        
        # Aggregate the output into a new dataframe:
        if year == start_year:
           hottest_load_output_df = hottest_load_subset_df
           hottest_temp_output_df = hottest_temp_subset_df
        else:
           hottest_load_output_df = pd.concat([hottest_load_output_df, hottest_load_subset_df], axis=1)
           hottest_temp_output_df = pd.concat([hottest_temp_output_df, hottest_temp_subset_df], axis=1)
    
    return hottest_load_output_df, hottest_temp_output_df


In [ ]:
# Test the function:
hottest_load_output_df, hottest_temp_output_df = process_heat_wave_load_time_series(load_data_input_dir = load_data_input_dir, 
                                                                                    temp_data_input_dir = temp_data_input_dir, 
                                                                                    ics_to_plot = 'WECC',
                                                                                    scenario = 'rcp45cooler_ssp5',
                                                                                    start_year = 2060,
                                                                                    end_year = 2100)
hottest_load_output_df


In [ ]:
# Define a function to plot the load evolution about the hottest hour of the year:
def plot_heat_wave_load_time_series(load_data_input_dir: str, temp_data_input_dir: str, ics_to_plot: str, scenario: str, start_year: int, end_year: int,
                                    image_output_dir: str, image_resolution: int, save_images=False):
    
    # Compute the time series matrix using the function created above:
    hottest_load_output_df, hottest_temp_output_df = process_heat_wave_load_time_series(load_data_input_dir = load_data_input_dir, 
                                                                                        temp_data_input_dir = temp_data_input_dir, 
                                                                                        ics_to_plot = ics_to_plot,
                                                                                        start_year = start_year,
                                                                                        end_year = end_year, 
                                                                                        scenario = scenario)
    
    # Make the plot:
    plt.figure(figsize=(20, 14))
    plt.rcParams['font.size'] = 15
    
    plt.subplot(211)
    plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), hottest_temp_output_df[str(1980)], color='r', linestyle='-', linewidth=0.5, label='40x Yearly Sample')
    plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), hottest_temp_output_df.mean(axis=1), color='k', linestyle='-', linewidth=3, label='Composite Mean')
    for year in range(start_year, end_year, 1):
        plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), hottest_temp_output_df[str(year)], color='r', linestyle='-', linewidth=0.5)
    plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), hottest_temp_output_df.mean(axis=1), color='k', linestyle='-', linewidth=5)
    plt.legend(loc='upper left', prop={'size': 18})   
    plt.xlim([-14, 14])
    plt.xticks(np.arange(-14.5, 14.5, 1, dtype=int),['-14','-13','-12','-11','-10','-9','-8','-7','-6','-5','-4','-3','-2','-1','$t_{0}$','+1','+2','+3','+4','+5','+6','+7','+8','+9','+10','+11','+12','+13','+14'])
    plt.ylim([hottest_temp_output_df.dropna().values.min()-2, hottest_temp_output_df.dropna().values.max()+2])
    plt.ylim([80, 105])
    plt.grid(True)
    # plt.xlabel('Days from the Hottest Temperature of the Year');
    plt.xlabel('');
    plt.ylabel('Daily Max Population-Weighted Temperature [$^\circ$F]')
    plt.title(('Heat Wave Temperature Evolution in the ' + ics_to_plot + ' from ' + str(start_year) + ' to ' + str(end_year) + ' (' + scenario + ')'), fontsize=21)
        
    plt.subplot(212)
    for year in range(start_year, end_year, 1):
        plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), hottest_load_output_df[str(year)], color='r', linestyle='-', linewidth=0.5)
    plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), hottest_load_output_df.mean(axis=1), color='k', linestyle='-', linewidth=5)
    plt.xlim([-14, 14])
    plt.xticks(np.arange(-14.5, 14.5, 1, dtype=int),['-14','-13','-12','-11','-10','-9','-8','-7','-6','-5','-4','-3','-2','-1','$t_{0}$','+1','+2','+3','+4','+5','+6','+7','+8','+9','+10','+11','+12','+13','+14'])
    plt.ylim([0.75, 1.05])
    plt.grid(True)
    plt.xlabel('Days from the Hottest Temperature of the Year');
    plt.ylabel('Peak Load Normalized to the Hottest Day')
    plt.title(('Heat Wave Load Evolution'), fontsize=21)
    
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       filename = (ics_to_plot + '_' + scenario + '_Heat_Wave_Load_Evolution.png')
       plt.savefig(os.path.join(image_output_dir, filename), dpi=image_resolution, bbox_inches='tight', facecolor='white')


In [ ]:
plot_heat_wave_load_time_series(load_data_input_dir = load_data_input_dir, 
                                temp_data_input_dir = temp_data_input_dir, 
                                ics_to_plot = 'WECC', 
                                scenario = 'historic', 
                                start_year = 1980,
                                end_year = 2020,
                                image_output_dir = image_output_dir, 
                                image_resolution = 300, 
                                save_images = True)


In [ ]:
# Define a function to plot the load evolution about the hottest hour of the year:
def plot_composite_heat_wave_load_time_series(load_data_input_dir: str, temp_data_input_dir: str, ics_to_plot: str, start_year: int, end_year: int, 
                                              image_output_dir: str, image_resolution: int, save_images=False):
    
    # Compute the time series matrices using the function created above:
    hottest_load_output_df, hottest_temp_output_df = process_heat_wave_load_time_series(load_data_input_dir = load_data_input_dir, temp_data_input_dir = temp_data_input_dir, 
                                                                                        ics_to_plot = ics_to_plot, scenario = 'historic', start_year = 1980, end_year = 2020)
    historic_temp = hottest_temp_output_df.mean(axis=1)
    historic_load = hottest_load_output_df.mean(axis=1)
    del hottest_load_output_df, hottest_temp_output_df
    
    hottest_load_output_df, hottest_temp_output_df = process_heat_wave_load_time_series(load_data_input_dir = load_data_input_dir, temp_data_input_dir = temp_data_input_dir, 
                                                                                        ics_to_plot = ics_to_plot, scenario = 'rcp45cooler_ssp5', start_year = start_year, end_year = end_year)
    rcp45cooler_temp = hottest_temp_output_df.mean(axis=1)
    rcp45cooler_load = hottest_load_output_df.mean(axis=1)
    del hottest_load_output_df, hottest_temp_output_df
    
    hottest_load_output_df, hottest_temp_output_df = process_heat_wave_load_time_series(load_data_input_dir = load_data_input_dir, temp_data_input_dir = temp_data_input_dir, 
                                                                                        ics_to_plot = ics_to_plot, scenario = 'rcp45hotter_ssp5', start_year = start_year, end_year = end_year)
    rcp45hotter_temp = hottest_temp_output_df.mean(axis=1)
    rcp45hotter_load = hottest_load_output_df.mean(axis=1)
    del hottest_load_output_df, hottest_temp_output_df
    
    hottest_load_output_df, hottest_temp_output_df = process_heat_wave_load_time_series(load_data_input_dir = load_data_input_dir, temp_data_input_dir = temp_data_input_dir, 
                                                                                        ics_to_plot = ics_to_plot, scenario = 'rcp85cooler_ssp5', start_year = start_year, end_year = end_year)
    rcp85cooler_temp = hottest_temp_output_df.mean(axis=1)
    rcp85cooler_load = hottest_load_output_df.mean(axis=1)
    del hottest_load_output_df, hottest_temp_output_df
    
    hottest_load_output_df, hottest_temp_output_df = process_heat_wave_load_time_series(load_data_input_dir = load_data_input_dir, temp_data_input_dir = temp_data_input_dir, 
                                                                                        ics_to_plot = ics_to_plot, scenario = 'rcp85hotter_ssp5', start_year = start_year, end_year = end_year)
    rcp85hotter_temp = hottest_temp_output_df.mean(axis=1)
    rcp85hotter_load = hottest_load_output_df.mean(axis=1)
    del hottest_load_output_df, hottest_temp_output_df
    
    # Make the plot:
    plt.figure(figsize=(20, 14))
    plt.rcParams['font.size'] = 15
    
    plt.subplot(211)
    plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), historic_temp, color='black', linestyle='-', label='historic (1980-2019)', linewidth=3)
    plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), rcp45cooler_temp, color='blue', linestyle='-', label=('rcp45cooler_ssp5 (' + str(start_year) + '-' + str(end_year) + ')'), linewidth=3)
    plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), rcp45hotter_temp, color='cyan', linestyle='-', label=('rcp45hotter_ssp5 (' + str(start_year) + '-' + str(end_year) + ')'), linewidth=3)
    plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), rcp85cooler_temp, color='orange', linestyle='-', label=('rcp85cooler_ssp5 (' + str(start_year) + '-' + str(end_year) + ')'), linewidth=3)
    plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), rcp85hotter_temp, color='red', linestyle='-', label=('rcp85hotter_ssp5 (' + str(start_year) + '-' + str(end_year) + ')'), linewidth=3)
    plt.legend(loc='upper left', prop={'size': 18})
    plt.xlim([-14, 14])
    plt.xticks(np.arange(-14.5, 14.5, 1, dtype=int),['-14','-13','-12','-11','-10','-9','-8','-7','-6','-5','-4','-3','-2','-1','$t_{0}$','+1','+2','+3','+4','+5','+6','+7','+8','+9','+10','+11','+12','+13','+14'])
    plt.ylim([80, 105])
    plt.grid(True)
    # plt.xlabel('Days from the Hottest Temperature of the Year')
    plt.xlabel('');
    plt.ylabel('Daily Max Population-Weighted Temperature [$^\circ$F]')
    plt.title(('Composite Mean Heat Wave Temperature Evolution in the ' + ics_to_plot + ': ' + str(start_year) + ' to ' + str(end_year)), fontsize=21)
        
    plt.subplot(212)
    plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), historic_load, color='black', linestyle='-', label='historic (1980-2019)', linewidth=3)
    plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), rcp45cooler_load, color='blue', linestyle='-', label=('rcp45cooler_ssp5 (' + str(start_year) + '-' + str(end_year) + ')'), linewidth=3)
    plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), rcp45hotter_load, color='cyan', linestyle='-', label=('rcp45hotter_ssp5 (' + str(start_year) + '-' + str(end_year) + ')'), linewidth=3)
    plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), rcp85cooler_load, color='orange', linestyle='-', label=('rcp85cooler_ssp5 (' + str(start_year) + '-' + str(end_year) + ')'), linewidth=3)
    plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), rcp85hotter_load, color='red', linestyle='-', label=('rcp85hotter_ssp5 (' + str(start_year) + '-' + str(end_year) + ')'), linewidth=3)
    plt.xlim([-14, 14])
    plt.xticks(np.arange(-14.5, 14.5, 1, dtype=int),['-14','-13','-12','-11','-10','-9','-8','-7','-6','-5','-4','-3','-2','-1','$t_{0}$','+1','+2','+3','+4','+5','+6','+7','+8','+9','+10','+11','+12','+13','+14'])
    plt.ylim([0.75, 1.05])
    plt.grid(True)
    plt.xlabel('Days from the Hottest Temperature of the Year');
    plt.ylabel('Peak Load Normalized to the Hottest Day')
    plt.title(('Composite Mean Heat Wave Load Evolution in the ' + ics_to_plot + ': ' + str(start_year) + ' to ' + str(end_year)), fontsize=21)
    
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       filename = (ics_to_plot + '_Composite_Heat_Wave_Load_Evolution_' + str(start_year) + '_' + str(end_year) + '.png')
       plt.savefig(os.path.join(image_output_dir, filename), dpi=image_resolution, bbox_inches='tight', facecolor='white')


In [ ]:
plot_composite_heat_wave_load_time_series(load_data_input_dir = load_data_input_dir, 
                                          temp_data_input_dir = temp_data_input_dir, 
                                          ics_to_plot = 'WECC', 
                                          start_year = 2060,
                                          end_year = 2100,
                                          image_output_dir = image_output_dir, 
                                          image_resolution = 300, 
                                          save_images = True)


In [ ]:
# Define a function to process the relationship between temperatures and loads:
def process_temperature_load_relationship(load_data_input_dir: str, temp_data_input_dir: str, ics_to_plot: str, scenario: str, start_year: int, end_year: int):
    
    # Process the data using the function created above:
    ics_df = process_merged_time_series(load_data_input_dir = load_data_input_dir, 
                                        temp_data_input_dir = temp_data_input_dir,
                                        ics_to_plot = ics_to_plot,
                                        scenario = scenario,
                                        start_year = start_year,
                                        end_year = end_year)
    
    # Set the 'Time_UTC' value as a datetime variable:
    ics_df['Time_UTC'] = pd.to_datetime(ics_df['Time_UTC'])
    
    # Add a column with the year values to be used in grouping:
    ics_df['Year'] = ics_df['Time_UTC'].dt.year
    
    #Initiate a counter and empty dataframe to store the results:
    counter = 0;
    stats_df = pd.DataFrame()
    
    # Loop over the years:
    for year in range(start_year, end_year, 1):
        # Iterate the counter by one:
        counter = counter + 1 
        
        # Subset to just the year in question:
        met_df = ics_df.loc[(ics_df['Year'] == year)].copy()
    
        # Find hottest hour of the year:
        hottest_day = pd.DataFrame(met_df.loc[met_df['T2'].idxmax()].copy())
        
        # Put the results in the stats dataframe:
        stats_df.loc[counter, 'Year'] = year
        stats_df.loc[counter, 'Time_UTC'] = met_df['Time_UTC'].loc[met_df['T2'].idxmax()]
        stats_df.loc[counter, 'T2'] = met_df['T2'].loc[met_df['T2'].idxmax()]
        stats_df.loc[counter, 'Load_MWh'] = met_df['Load_MWh'].loc[met_df['T2'].idxmax()]
        stats_df.loc[counter, 'Percentile_Load'] = met_df['Percentile_Load'].loc[met_df['T2'].idxmax()]
        stats_df.loc[counter, 'Normalized_Load'] = met_df['Normalized_Load'].loc[met_df['T2'].idxmax()]
    
    # Compute the regression between temperature and normalized load:
    x = stats_df['T2'].values
    y = stats_df['Normalized_Load'].values
    x = x.reshape(stats_df.shape[0], 1)
    y = y.reshape(stats_df.shape[0], 1)
    regr = linear_model.LinearRegression()
    regr.fit(x, y)
    slope = str((regr.coef_).round(3)).replace('[', '').replace(']', '')
    
    return stats_df, regr, slope


In [ ]:
# Test the function:
stats_df, regr, slope = process_temperature_load_relationship(load_data_input_dir = load_data_input_dir, 
                                                              temp_data_input_dir = temp_data_input_dir, 
                                                              ics_to_plot = 'WECC',
                                                              scenario = 'historic',
                                                              start_year = 1980,
                                                              end_year = 2020)

slope


In [ ]:
# Define a function to process the load time series about the hottest hour of the year:
def plot_temperature_load_relationship(load_data_input_dir: str, temp_data_input_dir: str, ics_to_plot: str, scenario: str, start_year: int, end_year: int, 
                                       image_output_dir: str, image_resolution: int, save_images=False):
    
    # Process the data using the function described above
    stats_df, regr, slope = process_temperature_load_relationship(load_data_input_dir = load_data_input_dir, 
                                                                  temp_data_input_dir = temp_data_input_dir, 
                                                                  ics_to_plot = ics_to_plot,
                                                                  scenario = scenario,
                                                                  start_year = start_year,
                                                                  end_year = end_year)
    x = stats_df['T2'].values
    x = x.reshape(stats_df.shape[0], 1)
        
    # Make the plot:
    f = plt.figure(figsize=(25, 10))
    plt.rcParams['font.size'] = 18
    
    plt.subplot(121)
    plt.scatter(stats_df['T2'], stats_df['Load_MWh'], s=100, c=stats_df['Year'])
    plt.colorbar()
    plt.xlim((stats_df['T2'].min()-1), (stats_df['T2'].max()+1))
    plt.xlabel('Annual Maximum Temperature [$^\circ$F]');
    plt.ylabel('Raw Peak Demand [MW]')
        
    plt.subplot(122)
    plt.scatter(stats_df['T2'], stats_df['Normalized_Load'], s=100, c=stats_df['Year'])
    plt.plot(x, regr.predict(x), color='black', linewidth=5, label=(('Slope = ' + str(slope) + ' per 1$^\circ$F')))
    plt.colorbar()
    plt.legend(loc='lower right', prop={'size': 21})
    plt.xlim((stats_df['T2'].min()-1), (stats_df['T2'].max()+1))
    plt.xlabel('Annual Maximum Temperature [$^\circ$F]');
    plt.ylabel('Annually Normalized Peak Demand [Std. Dev. from the Mean]')
    plt.suptitle((ics_to_plot + ' Temperature-Load Relationship: ' + scenario + ' (' + str(start_year) + ' to ' + str(end_year) + ')'), fontsize=24)
    
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       filename = (ics_to_plot + '_' + scenario + '_T_Load_Relationship_' + str(start_year) + '_' + str(end_year) + '_.png')
       plt.savefig(os.path.join(image_output_dir, filename), dpi=image_resolution, bbox_inches='tight', facecolor='white')
        
    # return stats_df


In [ ]:
plot_temperature_load_relationship(load_data_input_dir = load_data_input_dir, 
                                    temp_data_input_dir = temp_data_input_dir, 
                                    ics_to_plot = 'WECC',
                                    scenario = 'historic',
                                    start_year = 1980,
                                    end_year = 2020,
                                    image_output_dir = image_output_dir, 
                                    image_resolution = 300, 
                                    save_images = True)


In [ ]:
# Define a function to plot the load evolution about the hottest hour of the year:
def plot_composite_temperature_load_relationships(load_data_input_dir: str, temp_data_input_dir: str, ics_to_plot: str, start_year: int, end_year: int, 
                                                  image_output_dir: str, image_resolution: int, save_images=False):
    
    # Process the data using the function described above
    hist_stats_df, hist_regr, hist_slope = process_temperature_load_relationship(load_data_input_dir = load_data_input_dir, temp_data_input_dir = temp_data_input_dir, 
                                                                                 ics_to_plot = ics_to_plot, scenario = 'historic', start_year = 1980, end_year = 2020)
    hist_x = hist_stats_df['T2'].values; hist_x = hist_x.reshape(hist_stats_df.shape[0], 1)
    
    rcp45cooler_stats_df, rcp45cooler_regr, rcp45cooler_slope = process_temperature_load_relationship(load_data_input_dir = load_data_input_dir, temp_data_input_dir = temp_data_input_dir, 
                                                                                                      ics_to_plot = ics_to_plot, scenario = 'rcp45cooler_ssp5', start_year = start_year, end_year = end_year)
    rcp45cooler_x = rcp45cooler_stats_df['T2'].values; rcp45cooler_x = rcp45cooler_x.reshape(rcp45cooler_stats_df.shape[0], 1)
    
    rcp45hotter_stats_df, rcp45hotter_regr, rcp45hotter_slope = process_temperature_load_relationship(load_data_input_dir = load_data_input_dir, temp_data_input_dir = temp_data_input_dir, 
                                                                                                      ics_to_plot = ics_to_plot, scenario = 'rcp45hotter_ssp5', start_year = start_year, end_year = end_year)
    rcp45hotter_x = rcp45hotter_stats_df['T2'].values; rcp45hotter_x = rcp45hotter_x.reshape(rcp45hotter_stats_df.shape[0], 1)
    
    rcp85cooler_stats_df, rcp85cooler_regr, rcp85cooler_slope = process_temperature_load_relationship(load_data_input_dir = load_data_input_dir, temp_data_input_dir = temp_data_input_dir, 
                                                                                                      ics_to_plot = ics_to_plot, scenario = 'rcp85cooler_ssp5', start_year = start_year, end_year = end_year)
    rcp85cooler_x = rcp85cooler_stats_df['T2'].values; rcp85cooler_x = rcp85cooler_x.reshape(rcp85cooler_stats_df.shape[0], 1)
    
    rcp85hotter_stats_df, rcp85hotter_regr, rcp85hotter_slope = process_temperature_load_relationship(load_data_input_dir = load_data_input_dir, temp_data_input_dir = temp_data_input_dir, 
                                                                                                      ics_to_plot = ics_to_plot, scenario = 'rcp85hotter_ssp5', start_year = start_year, end_year = end_year)
    rcp85hotter_x = rcp85hotter_stats_df['T2'].values; rcp85hotter_x = rcp85hotter_x.reshape(rcp85hotter_stats_df.shape[0], 1)
    
    # Make the plot:
    plt.figure(figsize=(20, 14))
    plt.rcParams['font.size'] = 15
    
    plt.scatter(hist_stats_df['T2'], hist_stats_df['Normalized_Load'], s=25, c='black')
    plt.scatter(rcp45cooler_stats_df['T2'], rcp45cooler_stats_df['Normalized_Load'], s=25, c='blue')
    plt.scatter(rcp45hotter_stats_df['T2'], rcp45hotter_stats_df['Normalized_Load'], s=25, c='cyan')
    plt.scatter(rcp85cooler_stats_df['T2'], rcp85cooler_stats_df['Normalized_Load'], s=25, c='orange')
    plt.scatter(rcp85hotter_stats_df['T2'], rcp85hotter_stats_df['Normalized_Load'], s=25, c='red')
    plt.plot(hist_x, regr.predict(hist_x), color='black', linewidth=5, label=(('historic Slope = ' + str(hist_slope) + ' per 1$^\circ$F')))
    plt.plot(rcp45cooler_x, rcp45cooler_regr.predict(rcp45cooler_x), color='blue', linewidth=5, label=(('rcp45cooler_ssp5 Slope = ' + str(rcp45cooler_slope) + ' per 1$^\circ$F')))
    plt.plot(rcp45hotter_x, rcp45hotter_regr.predict(rcp45hotter_x), color='cyan', linewidth=5, label=(('rcp45hotter_ssp5 Slope = ' + str(rcp45hotter_slope) + ' per 1$^\circ$F')))
    plt.plot(rcp85cooler_x, rcp85cooler_regr.predict(rcp85cooler_x), color='orange', linewidth=5, label=(('rcp85cooler_ssp5 Slope = ' + str(rcp85cooler_slope) + ' per 1$^\circ$F')))
    plt.plot(rcp85hotter_x, rcp85hotter_regr.predict(rcp85hotter_x), color='red', linewidth=5, label=(('rcp85hotter_ssp5 Slope = ' + str(rcp85hotter_slope) + ' per 1$^\circ$F')))
    plt.legend(loc='best', prop={'size': 21})
    plt.xlabel('Annual Maximum Temperature [$^\circ$F]');
    plt.ylabel('Annually Normalized Peak Demand [Std. Dev. from the Mean]')
    plt.title((ics_to_plot + ' Temperature-Load Relationship: ' + str(start_year) + ' to ' + str(end_year)), fontsize=24)
    
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       filename = (ics_to_plot + '_Composite_Temperature_Load_Relationship_' + str(start_year) + '_' + str(end_year) + '.png')
       plt.savefig(os.path.join(image_output_dir, filename), dpi=image_resolution, bbox_inches='tight', facecolor='white')


In [ ]:
plot_composite_temperature_load_relationships(load_data_input_dir = load_data_input_dir, 
                                              temp_data_input_dir = temp_data_input_dir, 
                                              ics_to_plot = 'WECC', 
                                              start_year = 2060,
                                              end_year = 2100,
                                              image_output_dir = image_output_dir, 
                                              image_resolution = 300, 
                                              save_images = True)
